In [1]:
include("src\\ImportanceSampling.jl")
using ImportanceSampling
using Distributions

In [2]:
srand(2016);

In [3]:
using Seaborn

# use q

In [4]:
p = Normal(8, 1)
f!(r, x) = (r .= exp.(-x.^2/2))
μ_actual = exp(-16)/sqrt(2)

niters = 10_000
# q = Normal(4, sqrt(1/2))
q = MixtureDistribution([Normal(4, sqrt(1/2)), Normal(2, 2)], [9, 1])
X = rand(q, 1, niters)

is = ImportanceSampler(f!, 1, q, p=p)
cvis = CvImportanceSampler(f!, 1, q, p=p, use_q=true)
;

In [5]:
update!(is, X=X[:, 1_001:end])
updateβ!(cvis, X=X[:, 1:1_000])
update!(cvis, X=X[:, 1_001:end])
;

In [6]:
abs(mean(cvis) - μ_actual), abs(mean(is) - μ_actual)

(7.957438516600256e-9, 1.9308598312598014e-10)

In [7]:
var(cvis), var(is)

(1.6984583224354317e-47, 3.9754311543069156e-20)

# simple

In [8]:
g!s = [((r, x) -> r .= 2*x, [1.0])
    ((r, x) -> begin
        y = first(x)
        r[1] = 5*y.^2
        r[2] = 2*y.^3
        r
    end, [5/3, 1/2])]

2-element Array{Tuple{Function,Array{Float64,1}},1}:
 (#3, [1.0])         
 (#4, [1.66667, 0.5])

In [9]:
f!(r, x) = r .= x.^2 - 3*x

niters = 10_000
q = Uniform()
X = rand(q, 1, niters)

cvis = CvImportanceSampler(f!, 1, q, g!s=g!s, w=((_)->1.0))
is = ImportanceSampler(f!, 1, q, w=((_)->1.0))

μ_actual = -7/6 
;

In [10]:
update!(is, X=X)
updateβ!(cvis, X=X[:, 1:1_000])
update!(cvis, X=X[:, 1_001:end])
;

In [11]:
coeffs(cvis)

3×1 Array{Float64,2}:
 -1.5        
  0.2        
 -1.39776e-13

In [12]:
abs.(mean(cvis) - μ_actual) , abs.(mean(is) - μ_actual)

(0.0, 0.00611653791870892)

In [13]:
var(cvis), var(is)

(3.3162702688746085e-33, 3.4226467239699256e-5)